In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import shapely
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import rsutils.s2_grid_utils as s2_grid_utils
import rsutils.esa_download as esa_download

In [ ]:
ethiopia_shape_filepath = '../data/outputs/shapefiles/eth_adm_csa_bofedb_2021_shp/eth_admbnda_adm0_csa_bofedb_itos_2021.shp'

ethiopia_s2grids_esa_stats_filepath = '../../ethiopia/data/ethiopia_s2_grids_esa_stats.geojson'

In [ ]:
ethiopia_gdf = gpd.read_file(ethiopia_shape_filepath)
ethiopia_gdf.crs

In [ ]:
ethiopia_gdf

In [ ]:
os.path.exists(ethiopia_s2grids_esa_stats_filepath)

In [ ]:
ESA_WORKING_DIR = '../data/esa'

In [ ]:
65120//10

In [ ]:
overwrite = False

if not os.path.exists(ethiopia_s2grids_esa_stats_filepath) or overwrite:
    ethiopia_polygon = ethiopia_gdf['geometry'][0]

    ethiopia_s2_grids_gdf = s2_grid_utils.get_s2_grids_gdf(
        geojson_epsg_4326 = ethiopia_polygon.__geo_interface__,
        scale_fact=1.0,
    )

    ethiopia_s2_grids_overlay_gdf = gpd.overlay(ethiopia_s2_grids_gdf, ethiopia_gdf[['geometry']])

    ethiopia_esa_raster_stats_gdf = esa_download.generate_esa_raster_stats_gdf(
        shapes_gdf = ethiopia_s2_grids_overlay_gdf,
        id_col = 'id',
        years = [2020, 2021],
        working_dir = ESA_WORKING_DIR,
        # njobs=1
    )

    ethiopia_esa_raster_stats_gdf.to_file(ethiopia_s2grids_esa_stats_filepath)
else:
    ethiopia_esa_raster_stats_gdf = gpd.read_file(ethiopia_s2grids_esa_stats_filepath)

In [ ]:
ethiopia_esa_raster_stats_gdf

In [ ]:
preexisting_stats_gdf = gpd.read_file(ethiopia_s2grids_esa_stats_filepath)

In [ ]:
preexisting_stats_gdf.shape

In [ ]:
(preexisting_stats_gdf[
    preexisting_stats_gdf['id'].isin(ethiopia_esa_raster_stats_gdf['id'])
][[col for col in preexisting_stats_gdf.columns if col not in ['id', 'geometry']]] \
== ethiopia_esa_raster_stats_gdf[
    [col for col in preexisting_stats_gdf.columns if col not in ['id', 'geometry']]
]).all()

In [ ]:
ethiopia_esa_raster_stats_gdf

In [ ]:
esa_stats_cols_2020 = []
esa_stats_cols_2021 = []

for col in ethiopia_esa_raster_stats_gdf.columns:
    if '(2020)' in col:
        esa_stats_cols_2020.append(col)
    elif '(2021)' in col:
        esa_stats_cols_2021.append(col)

In [ ]:
mandatory_cols = ['id', 'geometry']

In [ ]:
ethiopia_esa_raster_stats_gdf.loc[ethiopia_esa_raster_stats_gdf[esa_stats_cols_2020].sum(axis=1).argmin()]

In [ ]:
scale = 5
aspect_ratio = 3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))

x = '40 (2020)'
min_threshold = 0

g = sns.histplot(
    data = ethiopia_esa_raster_stats_gdf[ethiopia_esa_raster_stats_gdf[x] > min_threshold],
    x = x,
)

In [ ]:
n_intervals = 5

x = '40 (2020)'
min_threshold = 0

q = np.linspace(0, 1, n_intervals + 1)
quantiles = ethiopia_esa_raster_stats_gdf[ethiopia_esa_raster_stats_gdf[x] > min_threshold][x].quantile(q)
quantiles

In [ ]:
ethiopia_esa_raster_stats_gdf[
    (ethiopia_esa_raster_stats_gdf[x] >= 114473)
    # & (ethiopia_esa_raster_stats_gdf[x] < 114473)
].shape[0]

In [ ]:
ethiopia_esa_raster_stats_gdf[ethiopia_esa_raster_stats_gdf[x] > min_threshold][x].max()

In [ ]:
(ethiopia_esa_raster_stats_gdf['40 (2020)'] > 200000).sum()

In [ ]:
cropland_pixel_count_df = ethiopia_esa_raster_stats_gdf['40 (2020)'].value_counts().sort_index(ascending=False).reset_index()

In [ ]:
cropland_pixel_count_df

In [ ]:
cropland_pixel_count_df['cumcount'] = cropland_pixel_count_df['count'].cumsum()

In [ ]:
scale = 5
aspect_ratio = 3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))

g = sns.scatterplot(
    data = cropland_pixel_count_df,
    x = '40 (2020)',
    y = 'cumcount'
)

In [ ]:
ethiopia_esa_raster_stats_gdf[
    ethiopia_esa_raster_stats_gdf['40 (2020)'] > 200_000
][['id', 'geometry', '40 (2020)']].sort_values(by='40 (2020)', ascending=False)#.to_file('./data/top_ethiopia_grids.geojson')

In [ ]:
365 // 20

In [ ]:
1747 * 18